In [1]:
import pathlib
import numpy as np

import re

from matplotlib import pyplot as plt

# import data_processing
# from data_processing import *

from tqdm.notebook import trange, tqdm

## util functions

In [2]:
def _extract_digit(pattern, text):
    """ helper for extract_bw_and_sf """
    try:
        match = int(re.search(pattern, text)[1])
    except:
        print('ERROR! unable to extract bw or sf')
        return 
    else:
        return match 
    
    
def extract_bw_and_sf(filename):
    """ extracts params from input filename """
    _bw_pattern, _sf_pattern = r'BW(\d)', r'SF(\d{1,})'
    
    bw_match = _extract_digit(_bw_pattern, filename)
    if bw_match is not None:
        bw_match = BW_val[bw_match - 1]
    
    sf_match = _extract_digit(_sf_pattern, filename)
    
    return bw_match, sf_match

def check_and_load_file(filepath):
    filepath = pathlib.Path(filepath)
    
    if not filepath.exists():
        print(f'ERROR! unable to find input file at:\n{filepath}')
        return
        
    return filepath

### Get Encoding Parameters

In [4]:
def get_and_set_encoding_params(filepath):
    BW_val = np.array([10.4e3, 15.6e3, 0, 0, 0, 0, 125e3, 250e3, 500e3])
    BW, SF = extract_bw_and_sf(DATA_FILE.name) 
    
    Fs = int(1e6)
    sampPerSym = np.round(((2**SF)/BW)*Fs)
    pcktLen = 30.25*sampPerSym

    print(f'BW: {BW} | SF: {SF} | Fs: {Fs} | Samples Per Symbol: {sampPerSym} | Packet Length: {pcktLen}')

In [ ]:
##### data file
DATA_FILE = pathlib.Path('LoRa_Recording_OTA/Location5/lora_BW8_SF11_915MHz_1Msps_L5.dat')

# BW and SF setup
BW_val = np.array([10.4e3, 15.6e3, 0, 0, 0, 0, 125e3, 250e3, 500e3])
BW, SF = extract_bw_and_sf(DATA_FILE.name)

Fs = int(1e6)
sampPerSym = np.round(((2**SF)/BW)*Fs)
pcktLen = 30.25*sampPerSym

print(f'BW: {BW} | SF: {SF} | Fs: {Fs} | Samples Per Symbol: {sampPerSym} | Packet Length: {pcktLen}')

### Load Data

In [5]:
def load_data(filepath):
    try:
        signal = np.fromfile(DATA_FILE, dtype=np.complex64)
    except Exception as exc:
        print(f'unable to load file:\n{exc}')
        return 
    else:
        return signal

In [ ]:
signal = np.fromfile(DATA_FILE, dtype=np.complex64)

### Check Data Structure

In [ ]:
start, stop = int(145.4e3), int(145.4e3+4096)

fft_s = np.abs(np.fft.fftshift(np.abs(np.fft.fft(signal[start:stop]))))

plt.subplot(2,1,1)
plt.plot(signal[start-2000:stop])
plt.subplot(2,1,2)
plt.plot(fft_s)

In [ ]:
real_s = np.real(signal)
real_s = np.abs(real_s[0:int(5e6)])
th = 0.001

In [ ]:
plt.plot(real_s[0:int(1e6)])

In [ ]:
#norm_s = np.array([
#    np.ceil(val) if val > th else np.floor(val) if val < -th else 0
#    for val in real_s
#])

norm_s = np.array([
    np.ceil(val) if val >= th else 0
    for val in real_s
])

In [ ]:
num_zeros = np.where(norm_s == 0.0)[0]
plt.plot(num_zeros)

In [ ]:
import more_itertools as mit
x = [list(j) for j in mit.consecutive_groups(sorted(list(set(num_zeros))))]

In [ ]:
len_x = np.array([len(item) for item in x])

In [ ]:
plt.plot(len_x)

In [ ]:
for i in range(numPackets):
    for j in range(numSymbols):
        sym = packets(k:k+sampPerSym)
        k += sampPerSym